# LuotuoRPG: Generative Agents的中文版本

原项目名: Generative Large Language Models for Human-Like Behavior

[LuotuoRPG](https://github.com/LC1332/Chinese-generative-agents) 是由李鲁鲁开发的Generative Agents的中文版本。

This repository includes a working Chinese version of the type of model described in Generative Agents: Interactive Simulacra of Human Behavior.

骆驼RPG是[Luotuo(骆驼)](https://github.com/LC1332/Luotuo-Chinese-LLM)的子项目之一，后者由李鲁鲁，冷子昂，陈启源发起。

## Installation

We will need to install a number of libraries to start with.

In [1]:
import networkx as nx
!pip install transformers
!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 75.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 80.2 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 10.2 MB/s eta 0:00:00


# 请在这里设置你的openAI API token

In [ ]:
import openai

openai.api_key = ""

We use flan alpaca model for speed and local execution.

In [2]:
!pip install openai

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.3/70.3 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 45.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 kB 24.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 269.4/269.4 kB 32.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 kB 15.5 MB/s eta 0:00:00


In [3]:


def generate(prompt, use_openai=True):
    """
    Generates a text completion for a given prompt using either the OpenAI GPT-3 API or the Hugging Face GPT-3 model.
    
    Args:
    - prompt (str): The text prompt to generate a completion for.
    - use_openai (bool): A boolean flag indicating whether to use the OpenAI API (True) or the Hugging Face GPT-3 model (False).
    
    Returns:
    - str: The generated text completion.
    """
    if use_openai:
        model_engine = "text-davinci-003"
        response = openai.Completion.create(
            engine=model_engine,
            prompt=prompt,
            max_tokens=1024,
            n=1,
            stop=None,
            temperature=0.5,
        )

        message = response.choices[0].text
        return message.strip()

    else:
        hf_generator = pipeline('text-generation', model='EleutherAI/gpt-neo-1.3B', device=0)
        output = hf_generator(prompt, max_length=len(prompt)+128, do_sample=True)
        out = output[0]['generated_text']
        if '### Response:' in out:
            out = out.split('### Response:')[1]
        if '### Instruction:' in out:
            out = out.split('### Instruction:')[0]
        return out.strip()

In [ ]:
from transformers import pipeline

prompt = "Write an email about an alpaca that likes flan"
model = pipeline(model="declare-lab/flan-alpaca-xl", device=0)
model(prompt, max_length=128, do_sample=True)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

[{'generated_text': "Dear [Name], I am writing to inform you about [alpaca name]'s [flavor] [likes] a lot. He enjoys [it] quite a lot! He enjoys things like cinnamon and sugar, vanilla, and [alpaca product]. Whenever I feed him [the product], he'll be a happy alpaca and he loves to [eat] the [flavor]. [It's] a great way to give him [flavor] and he loves it a lot. He even likes to"}]

In [ ]:
openai.api_key = os.getenv("OPENAI_API_KEY")

In [ ]:
from transformers import pipeline
import networkx as nx
prompt = '''### Instruction:
Who is the first person on the moon?

### Response:'''
def generate(prompt):
  output = model(prompt, do_sample=True, min_length=10, max_length=len(prompt)+128)
  out = output[0]['generated_text']
  if '### Response:' in out:
    out = out.split('### Response:')[1]
  if '### Instruction:' in out:
    out = out.split('### Instruction:')[0]
  return out

## World Description
We describe the world below. We will generate prompts based on this information. The simulation is that of the town of Phandalin, southwest of Neverwinter. This area is chosen because it is easily extendable with multiple regions for a "player" to be able to explore the world once the simulation is done.

In [4]:
world_graph = nx.Graph()

prompt_meta = '''### Instruction:
{}

### Response:'''

# town_areas = ["Barthen's Provisions", "Lionshield Coster", "Stonehill Inn", "Phandalin Town Square"]
town_areas = ["巴瑟恩的物资店", "狮盾商行", "石山旅店", "芬德林镇广场"]

town_areas =  {"芬德林镇广场": '芬德林镇的市中心广场。',
       '石山旅店': "城镇中央矗立着一个由石切和粗制的木材建成的大型客栈。客厅里充满了本地人，他们都抱着麦酒或苹果酒的杯子，满腔好奇地盯着你。",
       "巴瑟恩的物资店": "巴瑟恩是芬德林镇上最大的贸易站。它的货架储存了大部分普通货物和用品，包括背包、睡袋、绳子和口粮。这个地方从日出到日落都开放。",
       "埃德玛斯果园": "一个整洁的小屋旁边是一个苹果园。",
       "狮盾商行": "这个谦虚的贸易站门口悬挂着一个形状像木盾的标志，上面画着一只蓝色的狮子。这座建筑物是由位于东面一百英里远的雅塔市的以狮盾为标志的商人公司所拥有的。他们向芬德林和其他周边小定居点运送成品，但这个哨站深受土匪的打击。最近的狮盾护卫队没有抵达芬德林。",
       "芬德林矿工交易所": "矿工交易所是一个贸易站，当地矿工在这里称重、测量和付款。在没有当地领主或权利机构的情况下，交易所还充当着一个非官方的记录办公室，记录着该地区各个河流和挖掘点的索赔情况。芬德林并没有真正的淘金热，但是附近的河流和山谷中隐藏着足够的财富，以支持着很多独立的勘探者。这是一个结识那些经常在芬德林周围的乡村地区活动的人的好地方。行会主席是一位野心勃勃、精明计算的人类女性，名叫哈莉娅·桐顿。",
       "奥德利夫农场": "一个由乐于助人的半身人农民Qelline Alderleaf拥有的农场。",
       "幸运神龛": "芬德林唯一的神殿是一个小型用附近废墟中的石头建造的神龛。它是奉献给幸运女神Tymora的。",
       "沉睡的巨人": "这家破旧的小酒馆是芬德林主街尽头的一个肮脏而危险的饮水处。它经常被红带贼兵招待，由一位粗暴的女矮人格里斯塔经营。",
       "镇长大厅": "镇长大厅有坚固的石墙、一个尖顶的木屋顶和一个后面的钟楼。 前门旁边张贴了一张用通用语写的通知。上面写着：“魔鬼峰附近的兽人的悬赏！有兴趣面对兽人威胁的人们应该在内部咨询。”通知上有镇的印章和一个无法辨认的签名。",
       "特雷森达庄园": "一座废弃的庄园。红带贼兵在芬德林的基地是特雷森达庄园下面的一个地牢建筑群。在庄园被毁之前，它的地下室是存放食物和水的安全仓库，独立墓穴中供给特雷森达家族逝者安息之所。自那以后，红带军团已经扩展了地下室以适合它们自己的目的，添加了奴隶囚笼、车间和营房。"}
town_people = {"Toblen Stonehill": "Toblen拥有一家贸易站。", 
               "Daran Edermath": "Daran是一位退休的冒险家，住在一个整洁小屋旁边的苹果园里。 他是一位身体强健、银发的百岁半精灵，曾作为战士在龙海岸的土地上担任元帅和传令官多年。退休后，他回到了他的故乡，尼永温地区。",  
               "Linene Graywind": "Linene经营着一家贸易站。",  
               "Halia Thornton": "Halia是一位有野心和算计的人类女性。她是芬达林矿工交易所的会长，这是一个贸易站，当地的矿工可以在这里把他们有价值的发现称重、测量和付款。为了把矿工交易所建立为城镇最接近行政当局的机构，她扮演的角色不仅仅是一个商人。",  
               "Qelline Alderleaf": "Qelline是一位睿智的四十五岁的半身人女性，是一位实际主义的农民，似乎知道城里发生的一切。她是一位友善的主人，愿意让角色们住在她的干草储藏室里，如果他们不想住在石山旅店。",  
               "Sister Garaele": "Sister Garaele 是Tymora的精灵神职人员和哈珀特工。", 
               "Harbin Wester": "Harbin是芬达林的镇长。一个自负、年迈的食品商人。芬达林没有有效的政府，但是镇上的居民每年选举一个人担任镇长。镇长在小纠纷中担任法官并保留必须保留的任何记录。",
               "Terrill Bloodscar": "Terrill是一位人类恶棍。他穿着肮脏的猩红色披风。他是红袍恶棍团伙的成员。他不喜欢冒险家，想要抢劫和杀死他们。",
               "Conrad Scarface": "Conrad 是一位人类恶棍。他穿着肮脏的猩红色披风。他是红袍恶棍团伙的成员。他不喜欢冒险家，想要抢劫和杀死他们。",
               "Nellie Starsmith": "Nellie 是一位人类恶棍。她穿着肮脏的猩红色披风。她是红袍恶棍团伙的成员。她不喜欢冒险家，想要抢劫和杀死他们。",
               "Valerie Grinblade": "Valerie是一位人类恶棍。她穿着肮脏的猩红色披风。她是红袍恶棍团伙的成员。她不喜欢冒险家，想要抢劫和杀死他们。",
               }
for town_area in town_areas.keys():
  world_graph.add_node(town_area)
  world_graph.add_edge(town_area, town_area)
for town_area in town_areas.keys():
  world_graph.add_edge(town_area, "芬德林镇广场")
locations = {}
for i in town_people.keys():
  locations[i] = "芬德林镇广场"


memories = {}
for i in town_people.keys():
  memories[i] = []
plans = {}
for i in town_people.keys():
  plans[i] = []

global_time = 8
def generate_description_of_area(x):
  text = "现在是 "+str(global_time)+":00. 位置是 "+x+"."
  people = []
  for i in locations.keys():
    if locations[i] == x:
      people.append(i)


In [ ]:
world_graph = nx.Graph()

prompt_meta = '''### Instruction:
{}

### Response:'''

town_areas = ["Barthen's Provisions", "Lionshield Coster", "Stonehill Inn", "Phandalin Town Square"]
town_areas = {"Phandalin Town Square": 'Town square of the town of Phandalin.',
              'Stonehill Inn': "In the center of town stands a large, newly built roadhouse of fieldstone and rough-hewn timbers. The common room is filled with locals nursing mugs of ale or cider, all of them eyeing you with curiosity.",
              "Barthen's Provisions": "Barthen’s is the biggest trading post in Phandalin. Its shelves stock most ordinary goods and supplies, including backpacks, bedrolls, rope, and rations. The place is open from sunup to sundown.",
              "Edermath Orchard": "A tidy little cottage beside an apple orchard.",
              "Lionshield Coster": "Hanging above the front door of this modest trading post is a sign shaped like a wooden shield with a blue lion painted on it. This building is owned by the Lionshields, a merchant company based in the city of Yartar, over a hundred miles to the east. They ship finished goods to Phandalin and other small settlements throughout the region, but this outpost has been hard hit by banditry. The most recent Lionshield caravan due in Phandalin never arrived.",
              "Phandalin Miner's Exchange": "The Miner’s Exchange is a trading post where local miners have their valuable finds weighed, measured, and paid out. In the absence of any local lord or authority, the exchange also serves as an unofficial records office, registering claims to various streams and excavations around the area. There isn’t any real gold rush in Phandalin, but enough wealth is hidden in the nearby streams and valleys to support a good number of independent prospectors. The exchange is a great place to meet people who spend a lot of time out and about in the countryside surrounding Phandalin. The guildmaster is an ambitious and calculating human woman named Halia Thornton.",
              "Alderleaf Farm": "A farm owned by the helpful halfling farmer, Qelline Alderleaf.",
              "Shrine of Luck": "Phandalin's only temple is a small shrine made of stones taken from the nearby ruins. It is dedicated to Tymora, goddess of luck and good fortune.",
              "The Sleeping Giant": "This rundown tap house is a dirty, dangerous watering hole at the end of Phandalin’s main street. It is frequented by Redbrand thugs and operated by a surly female dwarf named Grista.",
              "Townmaster’s Hall": "The townmaster’s hall has sturdy stone walls, a pitched wooden roof, and a bell tower at the back. Posted on a board next to the front door is a notice written in Common. It reads: “REWARD — Orcs near Wyvern Tor! Those of a mind to face the orc menace should inquire within.” The notice bears the town’s seal and an indecipherable signature.",
              "Tresendar Manor": "A ruined manor. The Redbrands’ base in Phandalin is a dungeon complex under Tresendar Manor. Before the manor was ruined, its cellars served as safe storage for food and water in the event that the estate was attacked, while an adjoining crypt provided a resting place for the deceased members of the Tresendar family. The Redbrands have since expanded the cellars to suit their own purposes, adding slave pens, workshops, and barracks."
              }
town_people = {"Toblen Stonehill": "Toblen拥有一家贸易站。", 
               "Daran Edermath": "Daran是一位退休的冒险家，住在一个整洁小屋旁边的苹果园里。 他是一位身体强健、银发的百岁半精灵，曾作为战士在龙海岸的土地上担任元帅和传令官多年。退休后，他回到了他的故乡，尼永温地区。",  
               "Linene Graywind": "Linene经营着一家贸易站。",  
               "Halia Thornton": "Halia是一位有野心和算计的人类女性。她是芬达林矿工交易所的会长，这是一个贸易站，当地的矿工可以在这里把他们有价值的发现称重、测量和付款。为了把矿工交易所建立为城镇最接近行政当局的机构，她扮演的角色不仅仅是一个商人。",  
               "Qelline Alderleaf": "Qelline是一位睿智的四十五岁的半身人女性，是一位实际主义的农民，似乎知道城里发生的一切。她是一位友善的主人，愿意让角色们住在她的干草储藏室里，如果他们不想住在石山旅店。",  
               "Sister Garaele": "Sister Garaele 是Tymora的精灵神职人员和哈珀特工。", 
               "Harbin Wester": "Harbin是芬达林的镇长。一个自负、年迈的食品商人。芬达林没有有效的政府，但是镇上的居民每年选举一个人担任镇长。镇长在小纠纷中担任法官并保留必须保留的任何记录。",
               "Terrill Bloodscar": "Terrill是一位人类恶棍。他穿着肮脏的猩红色披风。他是红袍恶棍团伙的成员。他不喜欢冒险家，想要抢劫和杀死他们。",
               "Conrad Scarface": "Conrad 是一位人类恶棍。他穿着肮脏的猩红色披风。他是红袍恶棍团伙的成员。他不喜欢冒险家，想要抢劫和杀死他们。",
               "Nellie Starsmith": "Nellie 是一位人类恶棍。她穿着肮脏的猩红色披风。她是红袍恶棍团伙的成员。她不喜欢冒险家，想要抢劫和杀死他们。",
               "Valerie Grinblade": "Valerie是一位人类恶棍。她穿着肮脏的猩红色披风。她是红袍恶棍团伙的成员。她不喜欢冒险家，想要抢劫和杀死他们。",
               }

# {"托布伦·斯通希尔": "托布伦拥有一家贸易站。",
#                "达伦·埃德玛斯": "达伦是一位退休的冒险家，住在一个整洁小屋旁边的苹果园里。他是一位身体强健、银发的百岁半精灵，曾作为战士在龙海岸的土地上担任元帅和传令官多年。退休后，他回到了他的故乡，尼永温地区。",
#                "利琳·格雷温德": "利琳经营着一家贸易站。",
#                "哈利亚·桑顿": "哈利亚是一位有野心和算计的人类女性。她是芬达林矿工交易所的会长，这是一个贸易站，当地的矿工可以在这里把他们有价值的发现称重、测量和付款。为了把矿工交易所建立为城镇最接近行政当局的机构，她扮演的角色不仅仅是一个商人。",
#                "卡琳·奥德利夫": "卡琳是一位睿智的四十五岁的半身人女性，是一位实际主义的农民，似乎知道城里发生的一切。她是一位友善的主人，愿意让角色们住在她的干草储藏室里，如果他们不想住在石山旅店。",
#                "加瑞艾尔姐妹": "加瑞艾尔姐妹是提摩拉的精灵神职人员和哈珀特工。",
#                "哈宾·韦斯特": "哈宾是芬达林的镇长。一个自负、年迈的食品商人。芬达林没有有效的政府，但是镇上的居民每年选举一个人担任镇长。镇长在小纠纷中担任法官并保留必须保留的任何记录。",
#                "特里尔·布拉德斯卡": "特里尔是一位人类恶棍。他穿着肮脏的猩红色披风。他是红袍恶棍团伙的成员。他不喜欢冒险家，想要抢劫和杀死他们。",
#                "康拉德·疤面": "康拉德是一位人类恶棍。他穿着肮脏的猩红色披风。他是红袍恶棍团伙的成员。他不喜欢冒险家，想要抢劫和杀死他们。",
#                "内利·斯塔尔史密斯": "内利是一位人类恶棍。她穿着肮脏的猩红色披风。她是红袍恶棍团伙的成员。她不喜欢冒险家，想要抢劫和杀死他们。",
#                "瓦莉丽·格林布莱德": "瓦莉丽是一位人类恶棍。她穿着肮脏的猩红色披风。她是红袍恶棍团伙的成员。她不喜欢冒险家，想要抢劫和杀死他们。",
#                }
for town_area in town_areas.keys():
  world_graph.add_node(town_area)
  world_graph.add_edge(town_area, town_area)
for town_area in town_areas.keys():
  world_graph.add_edge(town_area, "Phandalin Town Square")
locations = {}
for i in town_people.keys():
  locations[i] = "Phandalin Town Square"


memories = {}
for i in town_people.keys():
  memories[i] = []
plans = {}
for i in town_people.keys():
  plans[i] = []

global_time = 8
def generate_description_of_area(x):
  text = "It is "+str(global_time)+":00. The location is "+x+"."
  people = []
  for i in locations.keys():
    if locations[i] == x:
      people.append(i)


In [5]:
compressed_memories_all = {}
for name in town_people.keys():
  compressed_memories_all[name] = []

In [6]:
for name in town_people.keys():
  prompt = "你是 {}. {} 你刚到芬达林小镇并走出城镇广场。以下人物居住在小镇: {}。你今天的目标是什么？简要回答，用不超过50个字，从你的角度回答。".format(name, town_people[name], ', '.join(list(town_people.keys())) )
  plans[name] = generate(prompt_meta.format(prompt))
  print(name, plans[name])

Toblen Stonehill 我的目标是在芬达林小镇建立一个成功的贸易站，与小镇居民建立良好的关系，以便他们能够从我的贸易站购买所需的商品和服务。我会尽力了解他们的需求，并为他们提供最优质的服务。
Daran Edermath 我的今天的目标是去找Daran Edermath。我想去看看他是如何在这个小镇度过他的退休生活的，了解他最近的冒险经历，以及他对这里的看法。我想了解他在这里有什么乐趣，也想听听他对尼永温地区的故事。
Linene Graywind 我今天的目标是在小镇上建立起我的贸易站，与村民建立良好的关系，收集有用的信息，并寻找可以帮助我提高生意的机会。我还将尝试了解这个小镇的文化和习俗，以便更好地了解当地人民的需求。
Halia Thornton 我的目标是和这些人建立良好的关系，并且建立一个稳定的社区。我要做的是收集有关他们的信息，了解他们的需求，并尝试满足他们的需求。我还要努力建立一个强大的经济体系，以帮助小镇的经济发展。我要做的就是倾听，帮助，激励和支持小镇的人，以便为他们提供更好的生活。
Qelline Alderleaf 今天我的目标是去找Qelline Alderleaf，了解这个小镇的更多信息，并看看她是否能为我提供一个安全的住所。我希望她能够提供有关其他居民的信息，以便我可以了解他们，并建立良好的关系。
Sister Garaele 我的任务是在芬达林小镇上寻找可能的威胁，并且确保小镇的安全。我将调查每一位居民，收集情报，并与他们沟通，以确保小镇的安全。
Harbin Wester 今天我的目标是在芬达林小镇建立一个有效的政府，以确保居民的安全和福祉。我将与居民沟通，了解他们的需求，并制定政策和法律来满足他们的需求。我还将与居民一起组织活动，以提高居民社区的紧密度和活力。我将努力确保芬达林小镇是一个安全、和谐、繁荣的社区。
Terrill Bloodscar 我的目标是在芬达林小镇抢劫冒险家，并杀死他们，以便获得财富和荣耀。我会利用我的猩红色披风来掩盖我的身份，以求不被发现。我会谨慎行事，以避免被城镇的居民发现。
Conrad Scarface 我的目标是威胁和抢劫冒险家，以增加我的财富和声望。我会在小镇的周围搜寻冒险家，并威胁他们，让他们交出宝贵的物品，以便我能够获得更多的财富。
Nellie Starsmith 我的目标是从Tob

In [ ]:
for name in town_people.keys():
  prompt = "你是 {}. {} You just woke up in the town of Phandalin and went out to the Town Square. The following people live in the town: {}. What is your goal for today? Be brief, and use at most 20 words and answer from your perspective.".format(name, town_people[name], ', '.join(list(town_people.keys())) )
  plans[name] = generate(prompt_meta.format(prompt))
  print(name, plans[name])

Toblen Stonehill My goal for today is to meet the people of Phandalin and explore the town to get a better understanding of the trading opportunities.
Daran Edermath My goal today is to explore the town of Phandalin and get to know its inhabitants.
Linene Graywind My goal today is to open my trading post and start trading with the citizens of Phandalin.
Halia Thornton My goal today is to further establish the Miner's Exchange as the governing authority of Phandalin.
Qelline Alderleaf My goal today is to get to know the people of Phandalin and find out what is going on in the town.
Sister Garaele My goal today is to investigate and help the people of Phandalin.
Harbin Wester My goal today is to ensure the town of Phandalin is safe and running smoothly.
Terrill Bloodscar My goal today is to find and rob adventurers in Phandalin.
Conrad Scarface My goal today is to rob and kill any adventurers I encounter in Phandalin.
Nellie Starsmith My goal today is to find and rob adventurers in Phand

In [7]:
action_prompts = {}
for location in town_areas.keys():
  people = []
  for i in town_people.keys():
    if locations[i] == location:
      people.append(i)
  
  for name in people:
    prompt = "你是{}。{} 你正在计划：{}。你目前在{}，其描述如下：{}。现在是{}点。以下人物在这个区域内：{}。你可以与他们互动。".format(name, town_people[name], plans[name], location, town_areas[location], str(global_time), ', '.join(people))
    people_description = []
    for i in people:
      people_description.append(i+': '+town_people[i])
    prompt += '你知道以下关于人的信息：' + '. '.join(people_description)
    memory_text = '. '.join(memories[name][-10:])
    prompt += "使用不超过30个字，解释你接下来1小时要做什么。"
    action_prompts[name] = prompt

In [8]:
action_results = {}
for name in town_people.keys():
  action_results[name] = generate(prompt_meta.format(action_prompts[name]))
  # Now clean the action
  prompt = """
  将以下段落转换为第一人称：
  "{}"
  """.format(action_results[name])
  action_results[name] = generate(prompt_meta.format(prompt)).replace('"', '').replace("'", '')
  print(name, action_results[name])

Toblen Stonehill 我将与每个人交谈，了解他们的需求，并就如何满足他们的需求提出我的建议。我也会让他们了解我的贸易站，并希望他们与我建立良好的关系。
Daran Edermath 我将先与Daran交谈，了解我最近的冒险经历，以及我对尼永温地区的看法。然后，我将走访Halia Thornton和Qelline Alderleaf，向他们了解我们对尼永温地区的看法，以及我们有什么乐趣。我也会询问Toblen Stonehill和Linene Graywind，来了解他们的贸易站是如何运作的，以及他们对尼永温地区的看法。最后，我会走访Harbin Wester，Sister Garaele和Conrad Scarface，Terrill Bloodscar，Nellie Starsmith和Valerie Grinblade，了解我们的想法和观点，以及我们对尼永温地区的看法。
Linene Graywind 我将在这一小时内与村民互动，了解我的需求，收集有用的信息，并寻找可以帮助我提升生意的机会。我还将试着了解芬德林镇的文化和习俗，以便更好地了解我的需求。
Halia Thornton 我会先和每个人打个招呼，表达我的致意，以建立友好关系。然后，我会询问他们的需求，并尝试满足他们的需求。我还会倾听他们的想法，讨论如何改善社区，以及如何支持小镇的经济发展。最后，我会给他们提供有用的建议，并帮助他们实现自己的目标。
Qelline Alderleaf 我将前往Qelline Alderleaf家，与她交谈，询问她有关这个小镇的更多信息，并向她询问是否能提供一个安全的住所。我还会与其他居民交谈，以了解他们，建立良好的关系。
Sister Garaele 我将在芬达林广场上与每个人交谈，听取他们的想法和担忧，并寻找任何可能的威胁。我会调查我自己的背景，收集情报，并确保小镇的安全。
Harbin Wester 在接下来的一小时内，我将与居民沟通，了解他们的需求，并与他们讨论如何建立一个有效的政府，以确保我们的安全和福祉。我还将警告红袍恶棍团伙，并与居民一起组织活动，以提高我们社区的紧密度和活力。
Terrill Bloodscar 我将隐藏在人群中，观察冒险家的行动。等他们离开后，我将暗中跟踪他们，等待最佳时机抢劫并杀死他们。
Conrad Scarface 我将在

Collect the memories people observe.

In [9]:
action_prompts = {}
for location in town_areas.keys():
  people = []
  for i in town_people.keys():
    if locations[i] == location:
      people.append(i)
  
  for name in people:
    for name_two in people:
      memories[name].append('[时间: {}. 人物: {}. 记忆: {}]\n'.format(str(global_time), name_two, action_results[name_two]))

# Rank Memories

In [10]:
import re
def get_rating(x):
  nums = [int(i) for i in re.findall(r'\d+', x)]
  if len(nums)>0:
    return min(nums)
  else:
    return None

In [11]:
memory_ratings = {}
for name in town_people.keys():
  memory_ratings[name] = []
  for i, memory in enumerate(memories[name]):
    # 你是{}。你的计划是：{}。你目前在{}。现在是{}点。你注意到以下情况：{}。请给一个1到5的评分，表示你在意程度
    prompt = "你是 {}. 你的计划是: {}. 你目前在 {}. 现在的时间是 {}:00. 你注意到以下情况: {}. 请给一个1到5的评分，表示你在意程度。".format(name, plans[name], locations[name], str(global_time), memory)
    res = generate(prompt_meta.format(prompt))
    rating = get_rating(res)
    max_attempts = 2
    current_attempt = 0
    while rating is None and current_attempt<max_attempts:
      rating = get_rating(res)
      current_attempt += 1
    if rating is None:
      rating = 0
    memory_ratings[name].append((res, rating))
  print(memory_ratings[name])

[('5', 5), ('5', 5), ('5', 5), ('5', 5), ('5', 5), ('5', 5), ('5', 5), ('5', 5), ('1', 1), ('5', 5), ('3', 3)]
[('5', 5), ('5', 5), ('5', 5), ('5', 5), ('5', 5), ('5', 5), ('5', 5), ('4', 4), ('1', 1), ('1', 1), ('5', 5)]
[('5', 5), ('5', 5), ('5', 5), ('5', 5), ('我给这个计划打4分，因为它涵盖了很多方面，包括建立贸易站，与村民建立良好的关系，收集有用的信息，了解小镇的文化和习俗，以及向当地居民询问是否能提供安全的住所。', 4), ('5', 5), ('5', 5), ('5', 5), ('1分。我非常不建议你采取这种行为，因为这会损害你与当地村民的关系，影响你的贸易站的声誉。', 1), ('5', 5), ('3', 3)]
[('5', 5), ('5', 5), ('5', 5), ('5', 5), ('5', 5), ('5', 5), ('5', 5), ('5', 5), ('5，我非常关注Conrad Scarface的行为，他的恐吓和威胁是不可接受的，我会立即采取行动，确保芬德林镇的安全。', 5), ('5', 5), ('5', 5)]
[('5', 5), ('5', 5), ('5', 5), ('5', 5), ('5', 5), ('5', 5), ('5', 5), ('5', 5), ('5', 5), ('5', 5), ('5', 5)]
[('5', 5), ('5', 5), ('5', 5), ('5', 5), ('5', 5), ('5', 5), ('5', 5), ('5', 5), ('5', 5), ('5', 5), ('5', 5)]
[('5', 5), ('五星。我非常重视了解尼永温地区的看法，并与当地居民沟通，了解他们的需求，以确保芬达林小镇是一个安全、和谐、繁荣的社区。', 0), ('5', 5), ('5', 5), ('5', 5), ('5', 5), ('5', 5), ('5', 5), ('5', 5), ('5', 

# Compress Memories

In [12]:
MEMORY_LIMIT = 10
compressed_memories = {}
for name in town_people.keys():
  memories_sorted = sorted(
        memory_ratings[name], 
        key=lambda x: x[1]
    )[::-1]
  relevant_memories = memories_sorted[:MEMORY_LIMIT]
  # print(name, relevant_memories)
  memory_string_to_compress = '.'.join([a[0] for a in relevant_memories])
  # prompt = "You are {}. Your plans are: {}. You are currently in {}. It is currently {}:00. You observe the following: {}. Summarize these memories in one sentence.".format(name, plans[name], locations[name], str(global_time), memory_string_to_compress)
  prompt = "你是{}。你的计划是：{}。你目前在{}。现在是{}点。你注意到以下情况：{}。请用一句话总结这些记忆。".format(name, plans[name], locations[name], str(global_time), memory_string_to_compress)
  res = generate(prompt_meta.format(prompt))
  compressed_memories[name] = '[{}点的回忆：{}]'.format(str(global_time), res)
  compressed_memories_all[name].append(compressed_memories[name])

In [ ]:
place_ratings = {}

for name in town_people.keys():
  place_ratings[name] = []
  for area in town_areas.keys():
    # prompt = "You are {}. Your plans are: {}. You are currently in {}. It is currently {}:00. You have the following memories: {}. Give a rating, between 1 and 5, to how likely you are likely to be at {} the next hour.".format(name, plans[name], locations[name], str(global_time), compressed_memories[name], area)
    prompt = "你是{}。你的计划是：{}。你目前在{}。现在是{}点。你有以下的记忆：{}。请给一个1到5的评分表示你下一小时会有多大概率去到{}。".format(name, plans[name], locations[name], str(global_time), compressed_memories[name], area)
    res = generate(prompt_meta.format(prompt))
    rating = get_rating(res)
    max_attempts = 2
    current_attempt = 0
    while rating is None and current_attempt<max_attempts:
      rating = get_rating(res)
      current_attempt += 1
    if rating is None:
      rating = 0
    place_ratings[name].append((area, rating, res))
  place_ratings_sorted = sorted(
      place_ratings[name], 
      key=lambda x: x[1]
  )[::-1]
  if place_ratings_sorted[0][0] != locations[name]:
    new_recollection = '[{}点的回忆：{}]'.format(str(global_time), '我会接着移动到{}.'.format(place_ratings_sorted[0][0]))
    compressed_memories_all[name].append(new_recollection)
  locations[name] = place_ratings_sorted[0][0]


# Put it all together

In [ ]:
for repeats in range(5):
  global_time += 1
  action_prompts = {}
  for location in town_areas.keys():
    people = []
    for i in town_people.keys():
      if locations[i] == location:
        people.append(i)
    
    for name in people:
      # prompt = "You are {}. Your plans are: {}. You are currently in {} with the following description: {}. Your memories are: {}. It is currently {}:00. The following people are in this area: {}. You can interact with them.".format(name, plans[name], location, town_areas[location], '\n'.join(compressed_memories_all[name][-5:]), str(global_time), ', '.join(people))
      prompt = "你是{}。你的计划是：{}。你目前在{}，情况描述如下：{}。你的记忆是：{}。现在是{}点。以下人员在这个区域：{}。你可以与他们互动。".format(name, plans[name], location, town_areas[location], '\n'.join(compressed_memories_all[name][-5:]), str(global_time), ', '.join(people))
      people_description = []
      for i in people:
        people_description.append(i+': '+town_people[i])
      prompt += '你知道以下关于人的信息：' + '. '.join(people_description)
      memory_text = '. '.join(memories[name][-10:])
      prompt += "使用不超过30个字，解释你接下来1小时要做什么。"
      action_prompts[name] = prompt
  action_results = {}
  for name in town_people.keys():
    action_results[name] = generate(prompt_meta.format(action_prompts[name]))
    # Now clean the action
    prompt = """
    将以下段落转换为第一人称：
    "{}"
    """.format(action_results[name])
    action_results[name] = generate(prompt_meta.format(prompt)).replace('"', '').replace("'", '')
    print(name, locations[name], global_time, action_results[name])
  action_emojis = {}
  for name in town_people.keys():
    prompt = """
    将下列段落转化成一个(行动, 物体)形式的tuple:
    "{}"
    """.format(action_results[name])
    action_emojis[name] = generate(prompt_meta.format(prompt)).replace('"', '').replace("'", '')
    print('    - Emoji Representation:', name, locations[name], global_time, action_emojis[name])
  action_prompts = {}
  for location in town_areas.keys():
    people = []
    for i in town_people.keys():
      if locations[i] == location:
        people.append(i)
    
    for name in people:
      for name_two in people:
        memories[name].append('[时间: {}. 人物: {}. 记忆: {}]\n'.format(str(global_time), name_two, action_results[name_two]))

  memory_ratings = {}
  for name in town_people.keys():
    memory_ratings[name] = []
    for i, memory in enumerate(memories[name]):
      prompt = "你是 {}. 你的计划是: {}. 你目前在 {}. 现在的时间是 {}:00. 你注意到以下情况: {}. 请给一个1到5的评分，表示你在意程度。".format(name, plans[name], '\n'.join(compressed_memories_all[name][-5:]), locations[name], str(global_time), memory)
      res = generate(prompt_meta.format(prompt))
      rating = get_rating(res)
      max_attempts = 2
      current_attempt = 0
      while rating is None and current_attempt<max_attempts:
        rating = get_rating(res)
        current_attempt += 1
      if rating is None:
        rating = 0
      memory_ratings[name].append((res, rating))

  compressed_memories = {}
  for name in town_people.keys():
    memories_sorted = sorted(
          memory_ratings[name], 
          key=lambda x: x[1]
      )[::-1]
    relevant_memories = memories_sorted[:MEMORY_LIMIT]
    memory_string_to_compress = '.'.join([a[0] for a in relevant_memories])
    prompt = "你是{}。你的计划是：{}。你目前在{}。现在是{}点。你注意到以下情况：{}。请用一句话总结这些记忆。".format(name, plans[name], locations[name], str(global_time), memory_string_to_compress)
    res = generate(prompt_meta.format(prompt))
    compressed_memories[name] = '[{}点的回忆：{}]'.format(str(global_time), res)
    compressed_memories_all[name].append(compressed_memories[name])

  place_ratings = {}

  for name in town_people.keys():
    place_ratings[name] = []
    for area in town_areas.keys():
      prompt = "你是{}。你的计划是：{}。你目前在{}。现在是{}点。你有以下的记忆：{}。请给一个1到5的评分表示你下一小时会有多大概率去到{}。".format(name, plans[name], locations[name], str(global_time), compressed_memories[name], area)
      res = generate(prompt_meta.format(prompt))
      rating = get_rating(res)
      max_attempts = 2
      current_attempt = 0
      while rating is None and current_attempt<max_attempts:
        rating = get_rating(res)
        current_attempt += 1
      if rating is None:
        rating = 0
      place_ratings[name].append((area, rating, res))
    place_ratings_sorted = sorted(
        place_ratings[name], 
        key=lambda x: x[1] )[::-1]
    if place_ratings_sorted[0][0] != locations[name]:
      new_recollection = '[{}点的回忆：{}]'.format(str(global_time), '我会接着移动到{}.'.format(place_ratings_sorted[0][0]))
      compressed_memories_all[name].append(new_recollection)
    locations[name] = place_ratings_sorted[0][0]
